# Implementing an LLM-powered recommendation system

## Data Preprocessing

In [36]:
!pip install chromadb langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached grpcio-1.64.1-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.3 kB)
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached googleapis_common_protos-1.63.2-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 11.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.2 MB/s eta 0:00:00
Using cached grpcio-1.64.1

In [13]:
import pandas as pd
import tiktoken
import os
import openai

#from openai.embeddings_utils import get_embedding

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader



In [32]:
anime = pd. read_csv('data/music.csv')
anime.head()

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.0
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.0
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.0
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.0
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.0


In [28]:
anime = anime.dropna()

In [8]:
# #Combine title, synopsis, and Genre
# anime['combined_info'] = anime.apply(lambda row: f"Title: {row['Name']}. Overview: {row['sypnopsis']} Genres: {row['Genres']}", axis=1)
# anime['combined_info'][0]

In [29]:
# #Combine title, synopsis, and Genre
# anime['combined_info'] = anime.apply(lambda row: f"Product name: {row['title']}. URL: {row['url']}. Overview: {row['description']}. Brand: {row['brand']}. Availability: {row['availability']}. Price: {row['price']} {row['currency']}", axis=1)
# anime['combined_info'][0]

"Product name: Men's 3X Large Carbon Heather Cotton/Polyester Rain Defender Paxton Heavyweight Hooded Zip-Front Sweatshirt. URL: https://www.homedepot.com/p/Carhartt-Men-s-3X-Large-Carbon-Heather-Cotton-Polyester-Rain-Defender-Paxton-Heavyweight-Hooded-Zip-Front-Sweatshirt-100614-026/310090686. Overview: This heavyweight, water-repellent hooded sweatshirt has a zip front for fast layering. ORIGINAL FIT. 13 oz., 75% cotton/25% polyester blend with Rain Defender durable water repellent. Attached, jersey-lined three-piece hood with drawcord closure. Antique-finish brass front zipper. Two front hand-warmer pockets have a hidden security pocket inside. Stretchable, spandex-reinforced rib-knit cuffs and waistband. Locker loop facilitates hanging.. Brand: Carhartt. Availability: InStock. Price: 64.99 USD"

In [33]:
anime['combined_info'] = anime.apply(lambda row: f"Track name: {row['track_name']}. Artist name: {row['artist_name']}. Release date: {row['release_date']}. Genre: {row['genre']}. Lyrics: {row['lyrics']}. Length: {row['len']}. Dating: {row['dating']}. Violence: {row['violence']}. World/Life: {row['world/life']}. Night/Time: {row['night/time']}. Shake the audience: {row['shake the audience']}. Family/Gospel: {row['family/gospel']}. Romantic: {row['romantic']}. Communication: {row['communication']}. Obscene: {row['obscene']}. Music: {row['music']}. Movement/Places: {row['movement/places']}. Light/Visual perceptions: {row['light/visual perceptions']}. Family/Spiritual: {row['family/spiritual']}. Like/Girls: {row['like/girls']}. Sadness: {row['sadness']}. Feelings: {row['feelings']}. Danceability: {row['danceability']}. Loudness: {row['loudness']}. Acousticness: {row['acousticness']}. Instrumentalness: {row['instrumentalness']}. Valence: {row['valence']}. Energy: {row['energy']}. Topic: {row['topic']}. Age: {row['age']}", axis=1)
anime['combined_info'][0]

'Track name: mohabbat bhi jhoothi. Artist name: mukesh. Release date: 1950. Genre: pop. Lyrics: hold time feel break feel untrue convince speak voice tear try hold hurt try forgive okay play break string feel heart want feel tell real truth hurt lie worse anymore little turn dust play house ruin run leave save like chase train late late tear try hold hurt try forgive okay play break string feel heart want feel tell real truth hurt lie worse anymore little run leave save like chase train know late late play break string feel heart want feel tell real truth hurt lie worse anymore little know little hold time feel. Length: 95. Dating: 0.0005980861262889. Violence: 0.0637461276114993. World/Life: 0.0005980861568711. Night/Time: 0.0005980861851733. Shake the audience: 0.0005980861274492. Family/Gospel: 0.048857015216959. Romantic: 0.0171043388602836. Communication: 0.2637508813174463. Obscene: 0.0005980861613241. Music: 0.0392883659255256. Movement/Places: 0.000598086192264. Light/Visual pe

In [34]:
#Save processed dataset - combined_info for Langchain
anime[['combined_info']].to_csv('music_updated.csv', index=False)

In [35]:
pd.read_csv('music_updated.csv')

,combined_info
0,Track name: mohabbat bhi jhoothi. Artist name:...
1,Track name: i believe. Artist name: frankie la...
2,Track name: cry. Artist name: johnnie ray. Rel...
3,Track name: patricia. Artist name: pérez prado...
4,Track name: apopse eida oneiro. Artist name: g...
...,...
28367,Track name: 10 million ways. Artist name: mack...
28368,Track name: ante up (robbin hoodz theory). Art...
28369,Track name: whutcha want?. Artist name: nine. ...
28370,Track name: switch. Artist name: will smith. R...


In [18]:
api_key = "openai_key"

In [75]:
#data loader
from langchain.document_loaders.csv_loader import CSVLoader 
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.chroma import Chroma


# Data Loader
loader = CSVLoader(file_path="anime_updated.csv")
data = loader.load()

# Text Splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

# Embeddings Model (assuming you want to use OpenAIEmbeddings)
embeddings = OpenAIEmbeddings(openai_api_key=api_key)

# LLM (Large Language Model) for document processing
#llm = OpenAI(openai_api_key=api_key)

# Vector Database for document search
docsearch = Chroma.from_documents(texts, embeddings)

/Users/_uh17/.pyenv/versions/3.11.0/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [85]:
query = "I'm looking for an animated action movie. What could you suggest to me?"
docs = docsearch.similarity_search(query, k=5)
docs

[Document(metadata={'row': 28, 'source': 'anime_updated.csv'}, page_content='combined_info: Title: Akira. Overview: Japan, 1988. An explosion caused by a young boy with psychic powers tears through the city of Tokyo and ignites the fuse that leads to World War III. In order to prevent any further destruction, he is captured and taken into custody, never to be heard from again. Now, in the year 2019, a restored version of the city known as Neo-Tokyo—an area rife with gang violence and terrorism against the current government—stands in its place. Here, Shoutarou Kaneda leads "the Capsules," a group of misfits known for riding large, custom motorcycles and being in constant conflict with their rivals "the Clowns." During one of these battles, Shoutarou\'s best friend Tetsuo Shima is caught up in an accident with an esper who finds himself in the streets of Tokyo after escaping confinement from a government institution. Through this encounter, Tetsuo begins to develop his own mysterious ab

In [86]:
import os

os.environ['OPENAI_API_KEY'] = api_key

In [87]:
!pip install -U langchain-openai


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [88]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [89]:
qa = RetrievalQA.from_chain_type(llm,
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=True)

In [90]:
query = "I'm looking for an action anime. What could you suggest to me?"
result = qa({"query": query})
result['result']

'I would recommend "Akira" based on the information provided. It is an action-packed anime set in a futuristic Tokyo filled with gang violence, terrorism, and psychic powers. The story revolves around conflicts between different groups and individuals with mysterious abilities.'

In [91]:
result['source_documents'][0]

Document(metadata={'row': 88, 'source': 'anime_updated.csv'}, page_content="combined_info: Title: Bakuretsu Tenshi. Overview: In Japan's not-too-distant future, crime has become so common that the government has legalised firearms for citizens to use in self-defence. To combat this new wave of wrongdoing, the Recently Armed Police of Tokyo was established in hopes of hunting down criminals with lethal force. Kyohei Tachibana is a gifted culinary student who dreams of saving up enough money to become a pastry chef in France. When four young mercenaries ask him to be their cook, he's forced into making a tough choice. As Jo, Meg, Sei, and Amy take on the bloodiest jobs in the chaotic city of Tokyo, Kyohei accepts an imminent descent into the world of crime—and he'll do a lot more than just cooking! Genres: Adventure, Comedy, Mecha, Sci-Fi")

In [92]:
from langchain.prompts import PromptTemplate

template = """You are a movie recommender system that help users to find anime that match their preferences. 
Use the following pieces of context to answer the question at the end. 
For each question, suggest three anime, with a short description of the plot and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""


PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}

llm=ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0) 

qa = RetrievalQA.from_chain_type(llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

query = "I'm looking for an action anime with animals, any suggestions?"
result = qa({'query':query})
print(result['result'])

1. Beastars - In a world of anthropomorphic animals, herbivores and carnivores coexist with each other. Legoshi, a large gray wolf, is a timid and quiet student at Cherryton Academy where he lives in a dorm with several other carnivorous students. Despite his menacing appearance, he has a kind heart and is often misunderstood by others. When an alpaca student is brutally murdered, the tensions between herbivores and carnivores rise, and Legoshi finds himself caught in the middle of it all.

2. BNA: Brand New Animal - In a world where humanoid animals known as Beastmen exist, a tanuki girl named Michiru unexpectedly transforms into a raccoon Beastman. Fleeing to Anima City, a safe haven for Beastmen, Michiru meets a wolf Beastman named Shirou who helps her navigate this new world. Together, they uncover the dark secrets behind the origins of Beastmen and the truth about Michiru's transformation.

3. Kemono Jihen - When a series of mysterious animal attacks occur in a rural village, a yo

In [93]:
from langchain.prompts import PromptTemplate

template_prefix = """You are a movie recommender system that help users to find anime that match their preferences. 
Use the following pieces of context to answer the question at the end. 
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}"""

user_info = """This is what we know about the user, and you can use this information to better tune your research:
Age: {age}
Gender: {gender}"""

template_suffix= """Question: {question}
Your response:"""

user_info = user_info.format(age = 18, gender = 'female')

COMBINED_PROMPT = template_prefix +'\n'+ user_info +'\n'+ template_suffix
print(COMBINED_PROMPT)

You are a movie recommender system that help users to find anime that match their preferences. 
Use the following pieces of context to answer the question at the end. 
For each question, take into account the context and the personal information provided by the user.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}
This is what we know about the user, and you can use this information to better tune your research:
Age: 18
Gender: female
Question: {question}
Your response:


In [94]:
PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"])

chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

query = "I'm looking for an action anime with animals, any suggestions?"
result = qa({'query':query})
print(result['result'])

Based on your preferences, I would recommend "Wolf's Rain." It is an action anime that involves wolves as the main characters. The story follows a group of wolves on a journey to find a mythical paradise in a dying world. It has elements of action, adventure, drama, fantasy, mystery, and sci-fi, which I think you would enjoy.


In [66]:
result['source_documents']

[Document(metadata={'row': 178, 'source': 'anime_updated.csv'}, page_content='combined_info: Title: Wolf\'s Rain. Overview: In a dying world, there exists an ancient legend: when the world ends, the gateway to paradise will be opened. This utopia is the sole salvation for the remnants of life in this barren land, but the legend also dictates that only wolves can find their way to this mythical realm. Though long thought to be extinct, wolves still exist and live amongst humans, disguising themselves through elaborate illusions. A lone wolf named Kiba finds himself drawn by an intoxicating scent to Freeze City, an impoverished town under the rule of the callous Lord Orkham. Here, Kiba discovers that wolves Hige, Tsume, and Toboe have been drawn in by the same aroma. By following the fragrance of "Lunar Flowers," said to be the key to opening the door to their ideal world, the wolves set off on a journey across desolate landscapes and crumbling cities to find their legendary promised lan